1. With the exercises/earthquakes.csv file, select all the earthquakes in Japan with a of 4.9 or greater using the mb magnitude type.



In [77]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:\Users\bharo\OneDrive\Documents\earthquakes.csv')
df.head()
filter =  df[(df['magType']==  'mb') & (df['mag'] >= 4.9) & (df["place"].str.contains("Japan"))]
df1 = filter[['magType','mag', 'place']]
df1.head()
            

,magType,mag,place
1563,mb,4.9,"293km ESE of Iwo Jima, Japan"
2576,mb,5.4,"37km E of Tomakomai, Japan"
3072,mb,4.9,"15km ENE of Hasaki, Japan"
3632,mb,4.9,"53km ESE of Hitachi, Japan"


2.Create bins for each full number of magnitude (for example, the first bin is (0, 1], the second is (1, 2], and so on) with the ml magnitude type and count how many are in each bin.

In [78]:
x = df.query ("magType == 'ml'") ['mag']
q = pd.cut(x, np.arange(0, 10)) 
r = q.value_counts()
r



(1, 2]    3105
(0, 1]    2207
(2, 3]     862
(3, 4]     122
(4, 5]       2
(5, 6]       1
(6, 7]       0
(7, 8]       0
(8, 9]       0
Name: mag, dtype: int64

3. Using the exercises/faang.csv file, group by the ticker and resample to monthly frequency. Aggregate the open and close prices with the mean, the high price with the max, the low price with the min, and the volume with the sum.

In [79]:
faang = pd.read_csv(r'C:\Users\bharo\OneDrive\Desktop\faang.csv',index_col = 'date', parse_dates = ['date'])
faang.index = pd.to_datetime(faang.index)

xs = faang. resample('M')

xs.agg(
    {
        'open': np.mean,
        'high': np.max,
        'low': np.min,
        'close': np.mean,
        'volume': np.sum
    }
)



,open,high,low,close,volume
date,,,,,
2018-01-31,602.985367,1472.58,161.5708,605.670330,1518822484
2018-02-28,630.379919,1528.70,147.9865,629.441008,1809270408
2018-03-31,659.370467,1617.54,149.0200,657.890927,2149239610
2018-04-30,630.976625,1638.10,150.5100,628.496051,1851273970
2018-05-31,669.764352,1635.00,162.7911,672.370756,1267635998
2018-06-30,720.324453,1763.10,178.7056,720.472016,1276967283
2018-07-31,747.627404,1880.05,166.5600,748.380326,1481677778
2018-08-31,770.316621,2025.57,170.2700,771.750114,1587875763
2018-09-30,778.857875,2050.50,158.8656,777.852428,1473582000


4. Build a crosstab with the earthquake data between the tsunami column and the magType column. Rather than showing the frequency count, show the maximum magnitude that was observed for each combination. Put the magnitude type along the columns.

In [80]:
#https://www.youtube.com/watch?v=dulyiY894nk
pd.crosstab(index=df.tsunami, columns=df.magType, values=df.mag, aggfunc ='mean')

magType,mb,mb_lg,md,mh,ml,ms_20,mw,mwb,mwr,mww
tsunami,,,,,,,,,,
0,4.529443,2.653333,1.315301,0.520833,1.219135,NaN,3.59,5.75,4.385714,5.454762
1,5.296296,NaN,NaN,NaN,4.420000,5.7,4.12,NaN,NaN,5.684615


5. Calculate the rolling 60-day aggregations of the OHLC data by ticker for the FAANG data. Use the same aggregations as exercise 3.

In [84]:
faang.groupby('ticker').rolling(60).agg(
    {
        'open': np.mean,
        'high': np.max,
        'low': np.min,
        'close': np.mean,
        'volume': np.sum
    }
)

open      high     low       close       volume
ticker date                                                             
AAPL   2018-01-02         NaN       NaN     NaN         NaN          NaN
       2018-01-03         NaN       NaN     NaN         NaN          NaN
       2018-01-04         NaN       NaN     NaN         NaN          NaN
       2018-01-05         NaN       NaN     NaN         NaN          NaN
       2018-01-08         NaN       NaN     NaN         NaN          NaN
...                       ...       ...     ...         ...          ...
NFLX   2018-12-24  306.018050  386.7999  233.68  303.239833  811001766.0
       2018-12-26  303.596050  386.7999  231.23  301.232167  818289623.0
       2018-12-27  301.500383  386.7999  231.23  299.134417  822148280.0
       2018-12-28  299.393050  380.9300  231.23  297.116750  824496849.0
       2018-12-31  297.420217  380.0000  231.23  295.293583  832207164.0

[1255 rows x 5 columns]

6.Create a pivot table of the FAANG data that compares the stocks. Put the ticker in the rows and and show the averages of the OHLC and volume traded data.

In [68]:
faang.pivot_table(index = 'ticker') # average is a default aggfunc

,close,high,low,open,volume
ticker,,,,,
AAPL,186.986218,188.906858,185.135729,187.038674,3.402145e+07
AMZN,1641.726175,1662.839801,1619.840398,1644.072669,5.649563e+06
FB,171.510936,173.615298,169.303110,171.454424,2.768798e+07
GOOG,1113.225139,1125.777649,1101.001594,1113.554104,1.742645e+06
NFLX,319.290299,325.224583,313.187273,319.620533,1.147030e+07


7. Calculate the Z-scores of Amazon's data (ticker: AMZN) using apply().

In [69]:
import scipy.stats as stats
fa = faang.query("ticker == 'AMZN'")
fa.loc[:,['open','high','low','close','volume']].apply(lambda x : stats.zscore(x)).head()

,open,high,low,close,volume
date,,,,,
2018-01-02,-2.373298,-2.397493,-2.261116,-2.299432,-1.160358
2018-01-03,-2.291352,-2.318952,-2.171594,-2.222280,-0.997677
2018-01-04,-2.207394,-2.266321,-2.089267,-2.194903,-1.031723
2018-01-05,-2.144501,-2.199037,-2.062395,-2.095604,-0.826494
2018-01-08,-2.051545,-2.077651,-1.951536,-2.005550,-0.537988


8. Create a dataframe with the following three columns: ticker, date, and event.
set the index to ['date', 'ticker']
Merge the data with the FAANG data using an outer join.

In [70]:
vents = pd.DataFrame({
    'ticker': 'FB',
    'date': pd.to_datetime(
         ['2018-07-25', '2018-03-19', '2018-03-20']
    ), 
    'event': [
         'Disappointing user growth announced after close.',
         'Cambridge Analytica story',
         'FTC investigation'
    ]
}).set_index(['date', 'ticker'])

faang.reset_index().set_index(['date', 'ticker']).join(
    events, how='outer'
).sample(10, random_state=0)

,,open,high,low,close,volume,event
date,ticker,,,,,,
2018-01-03,AAPL,169.2521,171.2337,168.6929,168.9578,29517899,NaN
2018-05-23,NFLX,329.0400,345.0000,328.0900,344.7200,10049147,NaN
2018-01-17,FB,179.2600,179.3200,175.8000,177.6000,27992376,NaN
2018-10-17,AMZN,1842.7900,1845.0000,1807.0000,1831.7300,5295177,NaN
2018-02-26,AMZN,1509.2000,1522.8400,1507.0000,1521.9500,4954988,NaN
2018-01-05,GOOG,1094.0000,1104.2500,1092.0000,1102.2300,1279123,NaN
2018-04-04,FB,152.0250,155.5600,150.5100,155.1000,49885584,NaN
2018-05-30,AMZN,1618.1000,1626.0000,1612.9300,1624.8900,2907357,NaN
2018-04-17,NFLX,329.6600,338.6200,323.7700,336.0600,33866456,NaN


In [71]:

d = {'ticker':'FB', 'date':pd.to_datetime (['2018-07-25', '2018-03-19', '2018-03-20']), 'event': ['Disappointing user growth announced after close.',
         'Cambridge Analytica story',
         'FTC investigation'
    ]}
df = pd.DataFrame(data=d)
df.set_index(['date', 'ticker'])


,,event
date,ticker,
2018-07-25,FB,Disappointing user growth announced after close.
2018-03-19,FB,Cambridge Analytica story
2018-03-20,FB,FTC investigation


In [72]:
df = pd.DataFrame({
    'ticker': 'FB',
    'date': pd.to_datetime(
         ['2018-07-25', '2018-03-19', '2018-03-20']
    ), 
    'event': [
         'Disappointing user growth announced after close.',
         'Cambridge Analytica story',
         'FTC investigation'
    ]
}).set_index(['date', 'ticker'])

faang.reset_index().set_index(['date', 'ticker']).join(
    df, how='outer'
)

open       high        low      close    volume event
date       ticker                                                            
2018-01-02 AAPL     166.9271   169.0264   166.0442   168.9872  25555934   NaN
           AMZN    1172.0000  1190.0000  1170.5100  1189.0100   2694494   NaN
           FB       177.6800   181.5800   177.5500   181.4200  18151903   NaN
           GOOG    1048.3400  1066.9400  1045.2300  1065.0000   1237564   NaN
           NFLX     196.1000   201.6500   195.4200   201.0700  10966889   NaN
...                      ...        ...        ...        ...       ...   ...
2018-12-31 AAPL     157.8529   158.6794   155.8117   157.0663  35003466   NaN
           AMZN    1510.8000  1520.7600  1487.0000  1501.9700   6954507   NaN
           FB       134.4500   134.6400   129.9500   131.0900  24625308   NaN
           GOOG    1050.9600  1052.7000  1023.5900  1035.6100   1493722   NaN
           NFLX     260.1600   270.1001   260.0000   267.6600  13508920   NaN

[1255 rows x 6 columns]

8.Use the transform() method on the FAANG data to represent all the values in terms of the first date in the data. To do so, divide all values for each ticker by the values of the first date in the data for that ticker. This is referred to as an index, and the data for the first date is the base. More information. When data is in this format, we can easily see growth over time. Hint: transform() can take a function name.

In [73]:
faang = faang.reset_index().set_index(['ticker', 'date'])
faang_index = (faang / faang.groupby(level='ticker').transform('first'))
faang

open     high        low    close    volume
ticker date                                                      
FB     2018-01-02   177.68   181.58   177.5500   181.42  18151903
       2018-01-03   181.88   184.78   181.3300   184.67  16886563
       2018-01-04   184.90   186.21   184.0996   184.33  13880896
       2018-01-05   185.59   186.90   184.9300   186.85  13574535
       2018-01-08   187.20   188.90   186.3300   188.28  17994726
...                    ...      ...        ...      ...       ...
GOOG   2018-12-24   973.90  1003.54   970.1100   976.22   1590328
       2018-12-26   989.01  1040.00   983.0000  1039.46   2373270
       2018-12-27  1017.15  1043.89   997.0000  1043.88   2109777
       2018-12-28  1049.62  1055.56  1033.1000  1037.08   1413772
       2018-12-31  1050.96  1052.70  1023.5900  1035.61   1493722

[1255 rows x 5 columns]